# Arabic Sentiment using Tensorflow

Text -> Numaric representation -> Model
# First Approach using Vector Space Model VSM
in two steps:
* document vector and remove stopword
* convert document vector to numbers by:
1. Term Frequency TF
2. Invers Document Frequency IDF

IDF=Log(N/DF)+1 Number of documents


first approach has some limitations:
1. dictionary size (dimensions) can became huge
2. context not preserved

the solution: using Universal Sentence Encoder (USE). it will be the second approach 

    Generating features from Text by: Vector space model (VSM)
    Including Inverse Document frequency (IDF)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
import operator
import math
from functools import reduce
from sklearn.model_selection import train_test_split
import os
import json


2023-09-13 23:37:48.813331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('5556-ar-reviews.csv')
df.head()

,label,text
0,0,على الرغم من بعض الاقتباسات الجميله للكتاب إلا...
1,0,قرئتها ويا ليتني لم اقرئها ............... قمة...
2,0,أستمتعت رغم المنغصات . الموقع رائع والإستقلالث...
3,1,للراحة عنوان . كل شي. لا شي
4,1,رائعة ومقبضة وحزينة ومبتكرة


In [3]:
df['label'].value_counts(normalize=True)

0    0.659467
1    0.340533
Name: label, dtype: float64

In [4]:
df = df.sample(frac = 1) 

# Process Data

In [5]:
stopwords = {'فإذا', 'أنى', 'بمن', 'حتى', 'لم', 'أنتما', 'هناك', 'تينك', 'بل', 'إي', 'عن', 'ولكن', 'وإذا', 'دون', 'إنا', 'إذن', 'بكم', 'حين', 'عند', 'هل', 'إلا', 'هاته', 'ذينك', 'اللواتي', 'كذا', 'لستما', 'هي', 'اللتان', 'أكثر', 'كلتا', 'لكن', 'ليستا', 'هكذا', 'عسى', 'إذ', 'إن', 'اللاتي', 'إذا', 'بهم', 'نحن', 'فيما', 'ذاك', 'بكن', 'بيد', 'لهن', 'هذي', 'كأي', 'ذوا', 'أي', 'كلاهما', 'هذين', 'أينما', 'كي', 'إليكن', 'ماذا', 'هيا', 'هنالك', 'بي', 'بما', 'تلكما', 'بعض', 'بهن', 'تين', 'ريث', 'على', 'غير', 'حيثما', 'كأن', 'بخ', 'هاتان', 'هاهنا', 'ما', 'هيهات', 'لدى', 'شتان', 'لسنا', 'كيفما', 'مع', 'ممن', 'كما', 'إنما', 'يا', 'عليه', 'لك', 'ذه', 'ذان', 'لهما', 'ليست', 'لنا', 'مه', 'أنتن', 'في', 'لولا', 'بس', 'لها', 'أقل', 'عليك', 'فلا', 'مهما', 'ليسا', 'ذين', 'ذات', 'كلما', 'ذا', 'ذو', 'فيه', 'تي', 'هنا', 'هاتين', 'ها', 'هم', 'ألا', 'لا', 'سوى', 'وإذ', 'كم', 'لست', 'حيث', 'إليكما', 'لوما', 'الذين', 'كلا', 'التي', 'كأين', 'ذواتي', 'لستم', 'هذا', 'فمن', 'ذلكم', 'وما', 'كيف', 'لكم', 'حاشا', 'بك', 'والذي', 'أن', 'لهم', 'لسن', 'ثمة', 'ذي', 'وإن', 'ومن', 'أيها', 'له', 'متى', 'بلى', 'اللتين', 'لستن', 'بكما', 'قد', 'كليكما', 'لكما', 'هلا', 'آي', 'لكنما', 'اللذين', 'اللائي', 'ذلكن', 'لاسيما', 'ذلك', 'مذ', 'اللتيا', 'هما', 'إليك', 'سوف', 'منها', 'والذين', 'أنتم', 'هاتي', 'لكي', 'اللذان', 'ذواتا', 'عما', 'فيها', 'إلى', 'تلك', 'كل', 'لي', 'هو', 'فيم', 'إليكم', 'بها', 'ذانك', 'إنه', 'هؤلاء', 'أولئك', 'إذما', 'بنا', 'من', 'خلا', 'ليسوا', 'ثم', 'لعل', 'وهو', 'نحو', 'أين', 'لئن', 'عدا', 'آه', 'كأنما', 'كليهما', 'الذي', 'لن', 'نعم', 'هذه', 'بهما', 'ليت', 'تلكم', 'أما', 'منذ', 'أو', 'هاك', 'بماذا', 'كذلك', 'أنا', 'آها', 'فإن', 'عل', 'منه', 'هيت', 'أف', 'أم', 'إيه', 'كيت', 'ته', 'لكيلا', 'ليس', 'مما', 'هذان', 'أنت', 'حبذا', 'ولو', 'أوه', 'إما', 'لو', 'بين', 'به', 'ولا', 'لما', 'بعد', 'هن', 'ذلكما', 'أولاء','و'}

maxDictionaryLength = 8000

def tokenize(sentence, isCreateDict=False):
    tmpTokens = sentence.lower().split()
    tokens = [token for token in tmpTokens if ((token not in stopwords) and (len(token)> 0)) ]
     
    if isCreateDict:
        for token in tokens:
            if token in dictionary_dict:
                dictionary_dict[token] += 1
            else:
                dictionary_dict[token] = 1
    documentTokens.append(tokens)
    return tokens


def getInverseDocumentFrequency(documentTokens, dictionary):
    return list(map(lambda word : 1 + math.log(len(documentTokens) / reduce(lambda acc,curr: (1 if (word in curr) else 0) + acc, documentTokens,0)),dictionary))


  
def encoder(sentence, dictionary, idfs):
    tokens = tokenize(sentence)
    tfs = getTermFrequency(tokens, dictionary)
    tfidfs = getTfIdf(tfs,idfs)
    return tfidfs


def getTermFrequency(tokens, dictionary):
    return  list(map(lambda token: reduce(lambda acc,curr : (acc + 1 if (curr == token) else acc), tokens,0), dictionary))



def getTfIdf(tfs, idfs):
    return [tf * idf for (tf,idf) in zip(tfs,idfs)]



**Sample Test** Code used in the slides ( Module : preparing data for machine learning model )

In [6]:
dictionary_dict = {}
documentTokens = []
testComments = ['للراحة عنوان . كل شي. لا شي', 'شيء جميل']

for comment in testComments:
    documentTokens.append(tokenize(comment,True))


dictionary = sorted(dictionary_dict, key=dictionary_dict.get, reverse=True)
idfs = getInverseDocumentFrequency(documentTokens, dictionary);

tfidfs = []

for comment in testComments:
    tfidfs.append(encoder(comment, dictionary, idfs))

print(dictionary_dict)
print(dictionary)
print(idfs)
print(tfidfs)

{'للراحة': 1, 'عنوان': 1, '.': 1, 'شي.': 1, 'شي': 1, 'شيء': 1, 'جميل': 1}
['للراحة', 'عنوان', '.', 'شي.', 'شي', 'شيء', 'جميل']
[1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454]
[[1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.6931471805599454, 1.6931471805599454]]


In [7]:
dictionary_dict = {}
documentTokens = []
df['tokens'] = df['text'].apply(lambda x : tokenize(x, True))

In [8]:
df.head()

,label,text,tokens
2500,0,كتاب بسيط و سريع القراءة ... و لكن للأسف ليس ب...,"[كتاب, بسيط, سريع, القراءة, ..., للأسف, بالمست..."
1351,1,الراحة والاستجمام . الموقع هائل الإطلالة ساحرة...,"[الراحة, والاستجمام, ., الموقع, هائل, الإطلالة..."
619,1,استثنائي. جدا جدا تعامل رقي والموقع ممتاز.,"[استثنائي., جدا, جدا, تعامل, رقي, والموقع, ممت..."
153,0,ضعيف جدا. . كل شي مو زين والخدمه الفدق جدا سيئ...,"[ضعيف, جدا., ., شي, مو, زين, والخدمه, الفدق, ج..."
5266,0,تجربة لا باس بها . موقع الفندق جميل. أرسلت للف...,"[تجربة, باس, ., موقع, الفندق, جميل., أرسلت, لل..."


In [9]:
dictionary = sorted(dictionary_dict, key=dictionary_dict.get, reverse=True)
dictionary = dictionary[:maxDictionaryLength]
print('Length of dictionary : {0}'.format(len(dictionary)))
print(dictionary[:10])

Length of dictionary : 8000
['.', '..', '،', 'جدا', 'ان', 'كان', 'فى', 'الرواية', 'الكتاب', '...']


In [10]:
idfs = getInverseDocumentFrequency(documentTokens, dictionary)
len(idfs)

8000

In [11]:
df['features'] = df['text'].apply(lambda x : encoder(x,dictionary, idfs))
df['features'].head()

2500    [1.7678645203751016, 25.213736084559926, 0.0, ...
1351    [3.535729040750203, 0.0, 0.0, 2.56473576646237...
619     [0.0, 0.0, 0.0, 5.1294715329247556, 0.0, 0.0, ...
153     [1.7678645203751016, 0.0, 0.0, 2.5647357664623...
5266    [1.7678645203751016, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: features, dtype: object

In [12]:
df_new = df['features'].apply(lambda x : pd.Series(x))
df_new['label'] = df['label']

# Train Test Split

In [13]:
train, test = train_test_split(df_new, test_size=0.2)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

3999 train examples
445 validation examples
1112 test examples


In [14]:
print(train.shape, test.shape, val.shape)

(3999, 8001) (1112, 8001) (445, 8001)


In [15]:
def df_to_dataset(dataframe, shuffle=True, batch_size=16):
    dataframe = dataframe.copy()
    labels = dataframe.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [16]:
batch_size = 100
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
numOfFeatures = len(dictionary)

# Build Model

In [18]:
def get_build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(numOfFeatures,)),
    tf.keras.layers.Dropout(0.3),    
    tf.keras.layers.Dense(15, activation='relu'),  
    tf.keras.layers.Dropout(0.3),   
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=['accuracy'])
    return model

In [19]:
model = get_build_model()
model.summary()
model.fit(train_ds,epochs=20 ,validation_data=val_ds)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                120015    
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_1 (Dense)             (None, 15)                240       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 120271 (469.81 KB)
Trainable params: 120271 (469.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
40/40

# Evaluate Model

In [20]:
model.evaluate(test_ds)

12/12 [==============================] - 0s 4ms/step - loss: 1.8240 - accuracy: 0.7257


[1.8240069150924683, 0.7257194519042969]

# Make Predictions

In [21]:
## make predictions
testComments = ['صباح الخيرات', 'للراحة عنوان . كل شي. لا شي']
tfidfs = []
for comment in testComments:
    tfidfs.append(encoder(comment, dictionary, idfs))
print(f'predicted probabliities : {model.predict(tfidfs)}')
print(f'predicted classes : {tf.round(model.predict(tfidfs))}')

1/1 [==============================] - 0s 91ms/step
predicted probabliities : [[0.02793481]
 [0.999867  ]]
1/1 [==============================] - 0s 31ms/step
predicted classes : [[0.]
 [1.]]


# Export Model

In [22]:
model.save('ar_reviews.h5')

/usr/local/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
# write dictionary and IDFs

with open('dictionary.json', 'w', encoding='utf-8') as outfile:
    json.dump(dictionary, outfile,  ensure_ascii=False, indent=4)

with open('idfs.json', 'w', encoding='utf-8') as outfile:
    json.dump(idfs, outfile, ensure_ascii=False, indent=4)

# sklearn.ensemble import RandomForestClassifier

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf_model = RandomForestClassifier(n_jobs=-1,n_estimators=180)
rf_model.fit(train.loc[:, train.columns != 'label'], train['label'])

RandomForestClassifier(n_estimators=180, n_jobs=-1)

In [26]:
rf_model.score(train.loc[:, train.columns != 'label'], train['label'])

0.9987496874218554

# Testing

In [27]:
rf_model.score(test.loc[:, test.columns != 'label'],test['label'])

0.7805755395683454